In [1]:
from aevb.config import Config
from aevb.nets_flax import FlaxMLPEncoder, FlaxMLPDecoder

In [2]:
config = Config.from_yaml('./configs.yaml', override=["mnist", "small"])
config.prettyprint()


Config:
seed:            0            (int)
dataset:                      (str)
jax.gpus:        [0]          (ints)
run.steps:       10000        (int)
run.eval_every:  100          (int)
data_shape:      [28, 28]     (ints)
latent_dim:      4            (int)
gen_hidden:      [128, 784]   (ints)
rec_hidden:      [128, 64]    (ints)
act:             relu         (str)
init.init:       normal       (str)
init.params:     [0.0, 0.01]  (floats)
opt.opt:         adagrad      (str)
opt.lr:          0.1          (float)


In [4]:
from typing import Iterable
from math import prod

def create_gen_rec_mlps(config):
    data_shape = config['data_shape']

    if isinstance(data_shape, Iterable):
        out = prod(data_shape)
    else:
        out = data_shape
    
    gen_model = FlaxMLPDecoder(out, config['gen_hidden'], config['act'])
    rec_model = FlaxMLPEncoder(config['latent_dim'], config['rec_hidden'], activation=config['act'])
    return gen_model, rec_model


gen_model, rec_model = create_gen_rec_mlps(config)
